In [161]:
# Importing the data 
import pandas as pd

df = pd.read_csv("39666.csv")

In [162]:
# Show Info About The Dataset
#df
df.head()
#df.shape

# Check For Nulls
#df.isnull().sum()

# Check For Duplicates
#df.duplicated()

,Date,Country,Category 1,Category 2,Value
0,2013-04-01,Egypt,Generated Power,Total,11308.0
1,2013-04-01,Egypt,Power Sold,Total,31.0
2,2013-04-01,Egypt,Purchased Power,Boot,1194.0
3,2013-04-01,Egypt,Purchased Power,Industrial Companies,4.0
4,2013-04-01,Egypt,Purchased Power,New & Renewable ( Winds + Solar Energy ),130.0


In [163]:
# Also There is no Full Duplicates but we must check for any other types
# Scince There is 12 category * 12 month so every year must have 144 row 
# Reforfmat the date into Year-Month-Day
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
# Then count rows in every year

year_counts = df["Year"].value_counts().sort_index()
year_counts

Year
2013    108
2014    144
2015    144
2016    144
2017    146
2018    144
2019    144
2020    120
2021    132
2022    144
2023    152
2024    134
2025     24
Name: count, dtype: int64

In [164]:
# Show Duplicated Values(Year and Month)
all_months = set(range(1, 13))

for year, group in df.groupby("Year"):
    months_present = set(group["Month"].unique())
    missing_months = all_months - months_present
    
    if len(months_present) != 12:
        print(f"Year {year} has {len(months_present)} months recorded")
        print(f"Missing months: {sorted(list(missing_months))}\n")
# We get from this the needed values to be deleted

Year 2013 has 9 months recorded
Missing months: [1, 2, 3]

Year 2020 has 10 months recorded
Missing months: [5, 6]

Year 2021 has 11 months recorded
Missing months: [5]

Year 2024 has 11 months recorded
Missing months: [8]

Year 2025 has 2 months recorded
Missing months: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]



In [167]:
# Show the Real Null values
month_counts = (
    df.groupby(["Year", "Month"])
    .size()
    .reset_index(name="Count")
    .sort_values(["Year", "Month"])
)

for _, row in month_counts.iterrows():
    if row["Count"] != 12:
        print(f"\nYear {int(row['Year'])}, Month {int(row['Month'])} has {row['Count']} values")



In [166]:
# Now we have clear vision of the Nulls and Duplicates
# Let's Clear the dataset
df = df.drop_duplicates(subset=["Year", "Month", "Value"])

In [168]:
# Show the Real Null values again
month_counts = (
    df.groupby(["Year", "Month"])
    .size()
    .reset_index(name="Count")
    .sort_values(["Year", "Month"])
)

for _, row in month_counts.iterrows():
    if row["Count"] != 12:
        print(f"\nYear {int(row['Year'])}, Month {int(row['Month'])} has {row['Count']} values")


# Zero Results! Duplicates are cleared



In [170]:
df["Category_Combined"] = df["Category 1"].fillna("") + ", " + df["Category 2"].fillna("")
df["Category_Combined"] = df["Category_Combined"].str.strip(", ")

counts = df["Category_Combined"].value_counts().reset_index()
counts.columns = ["Category_Combined", "Count"]

counts.to_csv("Categories.csv")

In [171]:
# There is Categories with differenet names only we will unify the naming
replace_map = {
    "Surplus Of Industrial Companies": "Industrial Companies",
    "New & Renewable ( Winds + Solar Energy )": "New & Renewable ( Wind + Solar Energy )",
    "Boot & International Electrical Interconnection Countries": "Boot",
    "International Electrical Interconnection Countries": "Boot"
}

df["Category 2"] = df["Category 2"].replace(replace_map)

In [172]:
# PIIIIIIIIIIIIIIIIIIIVOT
df["Category"] = df["Category 1"] + ", " + df["Category 2"]

# Pivot Table
pivot_df = df.pivot_table(
    index=["Date","Year", "Month", "Day", "Country"], 
    columns="Category", 
    values="Value",
    aggfunc="sum"   
).reset_index()


pivot_df.to_csv("pivot_categories.csv", index=False)


In [173]:
pivot_df.shape

(139, 17)

In [ ]:
# Clear names
pivot_df = pivot_df.rename(columns={
    "Date": "date",
    "Year": "year",
    "Month": "month",
    "Day": "day",
    "Country": "country",
    "Generated  Power, Total": "gen_power_total",
    "Power Sold, Total": "power_sold_total",
    "Purchased Power, Boot": "purchased_boot",
    "Purchased Power, Industrial Companies": "purchased_industrial",
    "Purchased Power, New & Renewable ( Wind + Solar Energy )": "purchased_renewable",
    "Purchased Power, Total": "purchased_total",
    "Total Generated & Purchased Power, Total": "total_supply",
    "Total Uses, Business": "use_business",
    "Total Uses, Household": "use_household",
    "Total Uses, Industrial": "use_industrial",
    "Total Uses, Other": "use_other",
    "Total Uses, Total": "use_total"
})


In [185]:
# Creating the schema(star)
# Dim_Date
dim_date = (
    pivot_df[["date","year","month","day"]]
    .drop_duplicates()
    .sort_values(["date"])
    .reset_index(drop=True)
)
dim_date["date_id"] = range(1, len(dim_date) + 1)

# Dim_Country
dim_country = (
    pivot_df[["country"]]
    .drop_duplicates()
    .sort_values(["country"])
    .reset_index(drop=True)
)
dim_country["country_id"] = range(1, len(dim_country) + 1)

# Dim_Use
use_cols = ["use_business","use_household","use_industrial","use_other","use_total"]
dim_use = (
    pivot_df[use_cols]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_use["use_id"] = range(1, len(dim_use) + 1)

# Dim_Purshed
purshed_cols = ["purchased_boot","purchased_industrial","purchased_renewable","purchased_total"]
dim_purshed = (
    pivot_df[purshed_cols]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_purshed["purshed_id"] = range(1, len(dim_purshed) + 1)

# Fact
fact_power = (
    pivot_df
    .merge(dim_date, on=["date","year","month","day"], how="left")
    .merge(dim_country, on="country", how="left")
    .merge(dim_use, on=use_cols, how="left")
    .merge(dim_purshed, on=purshed_cols, how="left")
)

fact_power = fact_power[[
    "total_supply","gen_power_total","power_sold_total",
    "country_id","date_id","use_id","purshed_id"
]].copy()

fact_power.insert(0, "ID", range(1, len(fact_power) + 1))


In [186]:
dim_date[["date_id","date","year","month","day"]].to_csv("dim_date.csv", index=False)
dim_country[["country_id","country"]].to_csv("dim_country.csv", index=False)
dim_use[["use_id"] + use_cols].to_csv("dim_use.csv", index=False)
dim_purshed[["purshed_id"] + purshed_cols].to_csv("dim_purshed.csv", index=False)
fact_power.to_csv("fact_power.csv", index=False)

In [ ]:
# Flag indicator on total_supply

def supply_flag(x):
    if x > 20000:
        return "High"
    elif 15000 < x <= 20000:
        return "Medium"
    else:
        return "Low"

fact_power["supply_flag"] = fact_power["total_supply"].apply(supply_flag)
fact_power.to_csv("fact_power.csv", index=False)